In [1]:
from sqlalchemy import create_engine
from pathlib import Path


p_departement = Path.cwd().parent.parent / "data" / "villes_france.db" 
engine = create_engine(f"sqlite:///{p_departement}", echo=True)

engine

Engine(sqlite:////home/kevin-desktop/Documents/Seafile/COURS/Advanced_Programming/data/villes_france.db)

In [2]:
from sqlalchemy import ForeignKey
from sqlalchemy import String, Text
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class Departement(Base):
    __tablename__ = "departement"
    departement_id: Mapped[int] = mapped_column(primary_key=True)
    departement_code: Mapped[str] = mapped_column(Text)
    departement_nom: Mapped[str] = mapped_column(Text)

    def is_outremer(self) -> bool:
        return len(self.departement_code) == 3

class Ville(Base):
    __tablename__ = "villes"
    
    id: Mapped[int] = mapped_column(primary_key=True)  # AUTOINCREMENT handled automatically
    department: Mapped[str] = mapped_column(ForeignKey("departement.departement_code"))  # Add FK
    name: Mapped[str] = mapped_column(Text)
    population_2012: Mapped[int] = mapped_column()
    surface: Mapped[float] = mapped_column()  # REAL maps to float
    commune_code: Mapped[str] = mapped_column(Text, unique=True)
    
    def __repr__(self) -> str:
        return f"Ville(name={self.name}, department={self.department}, pop={self.population_2012})"



In [3]:
from sqlalchemy.orm import Session
from sqlalchemy import select

session = Session(engine)


The **Session** is your main interface to the database:

- **Tracks objects**: Knows which objects are new, modified, or deleted
- **Manages transactions**: Groups operations together
- **Identity map**: Ensures one Python object per database row
- **Lazy loading**: Can fetch related data automatically

Think of it as your "conversation" with the database.

In [4]:
stmt = (
    select(Departement)
    .where(Departement.departement_code.startswith("9")) # All the departemens starting with 6
)


This is **class-level attribute access**:

- `Departement.departement_code` → References the column definition in your class
- `.startswith("6")` → SQLAlchemy translates this to SQL: `WHERE departement_code LIKE '6%'`
- **No SQL strings needed!** Pure Python expressions

In [5]:
ans = session.execute(stmt) # Returns Result object

for dept in ans.scalars():
    # dept is now a full Python object with all your class methods/attributes
    print(dept.departement_nom, dept.departement_code) # Access database columns, As Python attributes
    # If you had methods in your class, you could call them here too!
    print(f"The department {dept.departement_nom} is {'not ' if not dept.is_outremer() else ''}overseas.")

2025-08-29 22:56:19,981 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-29 22:56:19,984 INFO sqlalchemy.engine.Engine SELECT departement.departement_id, departement.departement_code, departement.departement_nom 
FROM departement 
WHERE (departement.departement_code LIKE ? || '%')
2025-08-29 22:56:19,985 INFO sqlalchemy.engine.Engine [generated in 0.00111s] ('9',)
Territoire de Belfort 90
The department Territoire de Belfort is not overseas.
Essonne 91
The department Essonne is not overseas.
Hauts-de-Seine 92
The department Hauts-de-Seine is not overseas.
Seine-Saint-Denis 93
The department Seine-Saint-Denis is not overseas.
Val-de-Marne 94
The department Val-de-Marne is not overseas.
Val-d'oise 95
The department Val-d'oise is not overseas.
Mayotte 976
The department Mayotte is overseas.
Guadeloupe 971
The department Guadeloupe is overseas.
Guyane 973
The department Guyane is overseas.
Martinique 972
The department Martinique is overseas.
Réunion 974
The department Réunion is ove

**What scalars() does:**

- **Input**: `Row(Departement(...))` - wrapped objects
- **Output**: `Departement(...)` - direct objects
- **Why**: Makes iteration cleaner and more Pythonic
The scalars() method unwraps these Row objects so you get your actual model objects directly:
Bottom line: scalars() removes the Row wrapper so you can work with your Python objects directly instead of having to use row[0] everywhere.

In [7]:
from sqlalchemy import func
#Let's say we want to know which overseas departement
# has the most cities of more than 500 inhabitants.
# why not...

stmt = (
    select(
        Departement,
        func.count(Ville.id).label('total_cities')
        )
    .join(Departement, Ville.department == Departement.departement_code)    
    .where(Ville.population_2012 > 500)
    .where(Departement.departement_code.startswith("97"))
    .group_by(Departement.departement_id)
    .order_by(func.count(Ville.id).desc())
    .limit(1)
    )

In [8]:
results = session.execute(stmt).fetchall()

2025-08-29 22:56:29,376 INFO sqlalchemy.engine.Engine SELECT departement.departement_id, departement.departement_code, departement.departement_nom, count(villes.id) AS total_cities 
FROM villes JOIN departement ON villes.department = departement.departement_code 
WHERE villes.population_2012 > ? AND (departement.departement_code LIKE ? || '%') GROUP BY departement.departement_id ORDER BY count(villes.id) DESC
 LIMIT ? OFFSET ?
2025-08-29 22:56:29,376 INFO sqlalchemy.engine.Engine [generated in 0.00059s] (500, '97', 1, 0)


In [9]:
for row in results:
    dept, count = row  # Unpack the tuple
    print(f"Department: {dept.departement_nom}, Cities > 500: {count}")



Department: Guadeloupe, Cities > 500: 34


In [10]:
# Or just the first result since we know we do limit 1:
dept, count = session.execute(stmt).fetchone()
print(f"Winner: {dept.departement_nom} with {count} cities")

2025-08-29 22:56:30,779 INFO sqlalchemy.engine.Engine SELECT departement.departement_id, departement.departement_code, departement.departement_nom, count(villes.id) AS total_cities 
FROM villes JOIN departement ON villes.department = departement.departement_code 
WHERE villes.population_2012 > ? AND (departement.departement_code LIKE ? || '%') GROUP BY departement.departement_id ORDER BY count(villes.id) DESC
 LIMIT ? OFFSET ?
2025-08-29 22:56:30,779 INFO sqlalchemy.engine.Engine [cached since 1.404s ago] (500, '97', 1, 0)
Winner: Guadeloupe with 34 cities


In [28]:
pop_dep = func.sum(Ville.population_2012)
square_pop = func.sum(Ville.population_2012*Ville.population_2012)
hhi_expr = 10000*square_pop/(pop_dep*pop_dep)

stmt = (
    select(
        Departement,
        hhi_expr.label("hhi")
    )
    .join(Departement, Ville.department == Departement.departement_code)
    .group_by(Departement.departement_id)
    .order_by(hhi_expr.desc())  # ← Réutiliser l'expression
    .limit(5)
)
res = session.execute(stmt).fetchall()


2025-08-29 23:17:42,691 INFO sqlalchemy.engine.Engine SELECT departement.departement_id, departement.departement_code, departement.departement_nom, (? * sum(villes.population_2012 * villes.population_2012)) / ((sum(villes.population_2012) * sum(villes.population_2012)) + 0.0) AS hhi 
FROM villes JOIN departement ON villes.department = departement.departement_code GROUP BY departement.departement_id ORDER BY (? * sum(villes.population_2012 * villes.population_2012)) / ((sum(villes.population_2012) * sum(villes.population_2012)) + 0.0) DESC
 LIMIT ? OFFSET ?
2025-08-29 23:17:42,692 INFO sqlalchemy.engine.Engine [cached since 56.74s ago] (10000, 10000, 5, 0)


In [34]:
for dep, hhi in res:
    print(f"Le département de {dep.departement_nom} à un hhi de {hhi:.0f}.")

Le département de Paris à un hhi de 10000.
Le département de Bouches-du-Rhône à un hhi de 1982.
Le département de Haute-Vienne à un hhi de 1465.
Le département de Territoire de Belfort à un hhi de 1364.
Le département de Haute-Garonne à un hhi de 1343.
